In [187]:
import pandas
import numpy as np
import sklearn

In [188]:
#data = pandas.read_csv('deltasSubstraction1.csv',sep=";|,", engine="python")

In [189]:
#data = pandas.read_csv('deltasSubstractionCards.csv',sep=";|,", engine="python")

In [190]:
data = pandas.read_csv('deltasSubstractionCardsContracts.csv',sep=";|,", engine="python")

In [191]:
#data = data.drop(['REPORT_MONTH','CLIENT_HASH_ID','GENDER','F1','AGE'],1)

In [192]:
#data = data.drop(['REPORT_MONTH','CLIENT_HASH_ID','GENDER','F1'],1).dropna()

In [193]:
data = data.drop(['REPORT_MONTH','CLIENT_HASH_ID'],1).dropna()

In [194]:
active = data.loc[data['ACTIVE_FLAG']==1].sample(n=22031,random_state = 1)
nonactive = data.loc[data['ACTIVE_FLAG']==0]
dataset = pandas.concat([active,nonactive])

In [195]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)

In [196]:
train = dataset.sample(n=22031,random_state = 7)
features = train.drop(['ACTIVE_FLAG'],1)
goals = train['ACTIVE_FLAG']
predict = dataset.sample(n=22031,random_state = 191)

In [165]:
#train = data
#features = train.drop(['ACTIVE_FLAG'],1)
#goals = train['ACTIVE_FLAG']
#predict = data

In [166]:
clf.fit(features, goals)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [167]:
truval = predict['ACTIVE_FLAG'].values
res = clf.predict(predict.drop(['ACTIVE_FLAG'],1))
tp = 0
fp = 0
tn = 0
fn = 0
for goal, predicted in zip(truval,res):
    if bool(goal):
        if predicted == 1:           
            tp += 1
            continue
        fn += 1
        continue
    if bool(predicted):
        fp += 1
        continue
    tn += 1

        
print(tp)
print(tn)
print(fp)
print(fn)
print("Полнота уходящих")
print(float(tn)/(tn+fp))

print("Полнота оставшихся")
print(float(tp)/(tp+fn))

print("Accuracy")
print(float(tp+tn)/(tp+fp+tn+fn))

prec = float(tp)/(tp+fp)
recall = float(tp)/(tp+fn)
print("Precision")
print(prec)
print("Recall")
print(recall)
print("Fmeasure")
print (2*prec*recall/(prec+recall))

8531
9358
1603
2539
Полнота уходящих
0.853754219506
Полнота оставшихся
0.77064137308
Accuracy
0.811992192819
Precision
0.84181961713
Recall
0.77064137308
Fmeasure
0.804659498208


In [168]:
predict.ACTIVE_FLAG.value_counts()

1    11070
0    10961
Name: ACTIVE_FLAG, dtype: int64

In [169]:
zip(list(dataset)[1:],clf.feature_importances_)

[('GOOD_PHONE', 0.019437187335952295),
 ('GOOD_EMAIL', 0.019524990190303244),
 ('AGE', 0.1454634424467775),
 ('TRN_CNT', 0.069511950146616749),
 ('TRN_CNT_POS', 0.068160616230748511),
 ('TRN_CNT_ATM', 0.057121444633761155),
 ('TRN_AMNT', 0.11822121793098835),
 ('TRN_AMNT_POS', 0.11375949960493958),
 ('TRN_AMNT_ATM', 0.11699385654953727),
 ('CONTRACT_TURN_AMOUNT', 0.12201750008110775),
 ('SALARY_INCOME', 0.058611590493123512),
 ('DBO_AUTH_WWW', 0.020008120942605415),
 ('DBO_AUTH_MB', 0.013521373287252189),
 ('DBO_OPER_CNT', 0.010169802451555025),
 ('DBO_REM_CNT', 0.0049892215214208365),
 ('DBO_PAY_CNT', 0.0085682492929698515),
 ('DBO_OPER_AMNT', 0.014170683928951152),
 ('DBO_REM_AMNT', 0.0082377453229773284),
 ('DBO_PAY_AMNT', 0.011511507608412356)]

In [170]:
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
res = gnb.fit(features, goals).predict(predict.drop(['ACTIVE_FLAG'],1))
tp = 0
fp = 0
tn = 0
fn = 0
for goal, predicted in zip(truval,res):
    if bool(goal):
        if predicted == 1:           
            tp += 1
            continue
        fn += 1
        continue
    if bool(predicted):
        fp += 1
        continue
    tn += 1

        
print(tp)
print(tn)
print(fp)
print(fn)
print("Полнота уходящих")
print(float(tn)/(tn+fp))

print("Полнота оставшихся")
print(float(tp)/(tp+fn))

print("Accuracy")
print(float(tp+tn)/(tp+fp+tn+fn))

prec = float(tp)/(tp+fp)
recall = float(tp)/(tp+fn)
print("Precision")
print(prec)
print("Recall")
print(recall)
print("Fmeasure")
print (2*prec*recall/(prec+recall))

1147
10515
446
9923
Полнота уходящих
0.959310281909
Полнота оставшихся
0.103613369467
Accuracy
0.529345013844
Precision
0.720025109856
Recall
0.103613369467
Fmeasure
0.181157703546


In [171]:
cls = sklearn.linear_model.LinearRegression(fit_intercept=True, normalize=False, copy_X=True, n_jobs=1)

In [172]:
clf.fit(features, goals)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [173]:
truval = predict['ACTIVE_FLAG'].values
r = clf.predict(predict.drop(['ACTIVE_FLAG'],1))
res=[]
for i in r:
    if i>0.5:
        res.append(1)
        continue
    res.append(0)
tp = 0
fp = 0
tn = 0
fn = 0
for goal, predicted in zip(truval,res):
    if bool(goal):
        if predicted == 1:           
            tp += 1
            continue
        fn += 1
        continue
    if bool(predicted):
        fp += 1
        continue
    tn += 1

        
print(tp)
print(tn)
print(fp)
print(fn)
print("Полнота уходящих")
print(float(tn)/(tn+fp))

print("Полнота оставшихся")
print(float(tp)/(tp+fn))

print("Accuracy")
print(float(tp+tn)/(tp+fp+tn+fn))

prec = float(tp)/(tp+fp)
recall = float(tp)/(tp+fn)
print("Precision")
print(prec)
print("Recall")
print(recall)
print("Fmeasure")
print (2*prec*recall/(prec+recall))

8547
9310
1651
2523
Полнота уходящих
0.84937505702
Полнота оставшихся
0.772086720867
Accuracy
0.810539694067
Precision
0.838105510884
Recall
0.772086720867
Fmeasure
0.803742712056


In [ ]:
dat = pandas.read_csv('deltasSubstractionCardsContracts.csv',sep=";|,", engine="python")
dat = data.drop(['REPORT_MONTH','CLIENT_HASH_ID','GENDER','F1'],1).dropna()
for clas in unique(data.F1):
    data = dat.loc[dat.F1 == clas]
    active = data.loc[data['ACTIVE_FLAG']==1].sample(n=22031,random_state = 1)
    nonactive = data.loc[data['ACTIVE_FLAG']==0]
    dataset = pandas.concat([active,nonactive])
    